# 2. Etapa de preprocesado de texto

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 🎯 Consideraciones para la función de extracción de features

Crearé una función reutilizable basada en lo siguiente:

- **Lematización**: útil para modelos simples como TF-IDF (reduce ruido), pero puede ser perjudicial en modelos complejos con embeddings porque pierden contexto y no es necesaria.

- **Texto en minúsculas**: indiferente para análisis de sentimientos, se normaliza por consistencia, sería interesante por las marcas, pero la práctica pide un análisis de sentimientos...

- **Signos de exclamación e interrogación**: importantes, se deben conservar ya que aportan tono emocional.

- **Stopwords y tokens no alfanuméricos**:
  - Ya se trató en el ejercicio 1, pero aquí se eliminan todos salvo aquellos con valor emocional como `not`, `no`, `never`, `why`, `how` o `what`.
  - También se conservan `!` y `?` por su aporte al tono del mensaje.
  - El resto de stopwords comunes y símbolos sin carga emocional se eliminan para reducir ruido.

- **Etiquetas gramaticales (POS tags)**:
  - Separarlas (`num_nouns`, `num_verbs`, etc.) es útil en modelos tradicionales porque detectan patrones específicos.
  - En modelos complejos (con embeddings), es mejor no separarlas: se reduce el riesgo de overfitting y el modelo ya entiende el contexto semántico.

- **Idioma**: Escojo y me arriesgo a que todo este en inglés, en un futuro y con más tiempo, quizás se podría inspeccionar más a fondo si por casualidad hay alguna palabra o texto que este en otro idioma

- **Rating**: Como la práctica especificaba el entrenamiento de modelos de clasificación binaria (no multiclase) supervisado, pues me decanto por el sistema tradicional de evaluación de exámenes, todo lo que es más o igual a 5 (aprobado, valoración positiva), y lo que es menos (suspenso, valoración negativa), si fuese de 0-5 estrellas el dataset, pues a partir de 2,5

In [6]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 43.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
import spacy
import json
import pandas as pd

# Cargar modelo spaCy
nlp = spacy.load("en_core_web_sm")

# Stopwords importantes para sentimiento
important_stopwords = {"not", "no", "never", "n't", "why", "how", "what"}
# Signos de puntuación que aportan tono emocional
important_punct = {"!", "?"}

# is_stop = token es un stop_word
# is_alpha = token tiene caracteres solo (ní simbolos, ni caracteres especiales, etc)
# is_punct = token es una puntuacion
def preprocess_text_sentiment(doc):
    tokens = []
    for token in doc:
        if token.is_space:
            continue
        if token.is_stop and token.text.lower() not in important_stopwords:
            continue
        if token.is_punct and token.text not in important_punct:
            continue
        if token.is_alpha or token.text in important_punct:
            tokens.append(token.lemma_.lower())
    return " ".join(tokens)

# Estas funciones de este bloque de código, las pasaré a un archivo llamado features.py para poder ser importadas
# por Ejercicio3.ipynb sin muchas complicaciones
def extract_sentiment_features(json_path):
    with open(json_path, mode='r', encoding='utf-8') as f:
        data = json.load(f)

    processed = []

    for item in data:
        text = item["review_text"]
        rating = int(item["rating"])
        doc = nlp(text)  # tokeniza el texto
        cleaned_text = preprocess_text_sentiment(doc)

        features = {
            # Texto limpio para TF-IDF, embeddings, etc.
            "clean_text": cleaned_text,

            # Características lingüísticas
            "review_length": len(text),
            "num_tokens": len(doc),
            "num_sentences": len(list(doc.sents)),
            "num_nouns": sum(1 for token in doc if token.pos_ == "NOUN"),
            "num_verbs": sum(1 for token in doc if token.pos_ == "VERB"),
            "num_adjectives": sum(1 for token in doc if token.pos_ == "ADJ"),
            "num_adverbs": sum(1 for token in doc if token.pos_ == "ADV"),
            "has_exclamation": int("!" in text),
            "has_question": int("?" in text),
            "rating": 1 if rating >= 5 else 0  # 0-4 Negativo|0 , 5-10 Positivo|1
        }

        processed.append(features)

    return pd.DataFrame(processed)